In [142]:
import numpy as np
import pandas as pd
from selenium.webdriver import Chrome, ActionChains
from selenium.common.exceptions import NoSuchElementException 
import time
import datetime

In [143]:
# Creating dataframe, for the parsed data
cols = ['full_title', 'guest', 'length', 'views', 'likes', 'dislikes', 'date', 'comments']
data = []
df = pd.DataFrame(data = data, columns = cols)

In [144]:
# Link on the page for all the videos
all_Videos = 'https://www.youtube.com/c/vdud/videos'

In [145]:
#Starting browser
driver = Chrome('C:/WebDriver/chromedriver')
driver.get(all_Videos)
driver.maximize_window()

In [146]:
# Function to load all the videos
def scrollDown():
    driver.execute_script("window.scrollTo(0, 1660)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 3200)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 5000)")
    time.sleep(2)

In [147]:
scrollDown()

In [148]:
# Collect all the videos and get the list of links
play_list = driver.find_elements_by_id('thumbnail')

hrefs = []
for video in play_list:
    hrefs.append(video.get_attribute('href'))

In [149]:
# It appears that some links were collected twice
hrefs = list(set(hrefs))
len(hrefs)

118

In [150]:
# Check if the advertisment button exists
def check_if_btn_exists():
    try:
        driver.find_element_by_class_name('ytp-ad-skip-button-text')
    except NoSuchElementException:
        return False
    return True

In [151]:
# Getting length of the video in minutes
# If statement on the len of the string in order not to get length of the advertising video
def getLength():
    length = driver.find_element_by_class_name('ytp-time-duration').text
    
    if check_if_btn_exists() \
    and driver.find_element_by_class_name('ytp-ad-skip-button-text').text in ('Пропустить рекламу', 'Пропустить'):
        driver.find_element_by_class_name('ytp-ad-skip-button-text').click()
    
    if len(length)<5:
        time.sleep(1.5)
        return getLength()
    elif len(length.split(':')) == 3:
        return int(length.split(':')[0])*60 + int(length.split(':')[1])
    else:
        return int(length.split(':')[0])

In [152]:
# Getting full title and the name of the guest
def getTitle():
    full_title = driver.title
    
    title_end = full_title.find(' / ')
    if title_end < 0:
        title_end = full_title.find('.')
        if title_end < 0:
            title_end = full_title.find('(') - 1
            
    full_title = full_title[0:title_end]
    if full_title.find(' - ') > 0:
        guest = full_title[0:full_title.find('-')]
    elif full_title.find(' — ') > 0:
            guest = full_title[0:full_title.find('—')]
    elif full_title.find(' – ') > 0:
        guest = full_title[0:full_title.find('–')]
    elif full_title.find('. ') > 0:
            guest = full_title[0:full_title.find('.')]
    elif full_title.find(': ') > 0:
            guest = full_title[0:full_title.find(':')]
    else:
        guest = full_title
    return [full_title, guest]

In [153]:
# Getting statistics information about the video
def getVideoStats():
    views = driver.find_element_by_class_name('yt-view-count-renderer').text
    views = int(views[0: views.find('п')-1].replace(' ', ''))
    likesDislikes = driver.find_element_by_id('top-level-buttons').text
    listLikDis = likesDislikes.replace(',', '.').split('\n')
    
    if listLikDis[0].find('ТЫС.') > 0:
        likes = float(listLikDis[0].split()[0])*1000
    elif listLikDis[0].find('МЛН') > 0:
        likes = float(listLikDis[0].split()[0])*1000000
        
    if listLikDis[1].find('ТЫС.') > 0:
        dislikes = float(listLikDis[1].split()[0])*1000
    elif listLikDis[1].find('МЛН') > 0:
        dislikes = float(listLikDis[1].split()[0])*1000000 
        
    return [views, likes, dislikes]

In [154]:
# Getting date of the interview
def getDate():
    date = driver.find_element_by_id('date').text.split()
    day = int(date[0][1:])
    
    months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    for i, mon in enumerate(months):
        if mon == date[1][:3]:
            month = i+1
            
    year = int(date[2])
    return datetime.datetime(year, month, day)

In [155]:
# Getting the amount of comments
def getComms():
    driver.execute_script("window.scrollTo(0, 880)")
    time.sleep(2)
    coms = driver.find_element_by_class_name('count-text').text
    return int(coms.split()[0])*1000 + int(coms.split()[1])

In [156]:
# Collecting all the information and updating the dataframe
def dfAppend():
    global df
    item = [getTitle()[0], getTitle()[1], getLength(), getVideoStats()[0], getVideoStats()[1], 
          getVideoStats()[2], getDate(), getComms()]
    series = pd.Series(item, index=df.columns)
    df = df.append(series, ignore_index=True)

In [157]:
# Going through the collected links
def goHrefs():
    for href in hrefs:
        if type(href) == str:
            driver.get(href)
            time.sleep(2)
            dfAppend()

In [158]:
t = time.time()
goHrefs()
print('Time to get all the data: {} mins'.format(round((time.time() - t) / 60, 2)))

Time to get all the data: 14.63 mins


In [159]:
df.shape

(117, 8)

In [160]:
df.head()

,full_title,guest,length,views,likes,dislikes,date,comments
0,"Долгополов - депрессия, политика, секс втроем",Долгополов,84,7461608,304000.0,36000.0,2019-11-25,40781
1,Егор Крид - уход из Black Star и звонок Попере...,Егор Крид,113,22334678,634000.0,50000.0,2019-05-28,68999
2,"Познер - о цензуре, страхе и Путине",Познер,78,12882333,338000.0,17000.0,2017-10-18,40944
3,Беслан. Помни,Беслан,190,21968899,1100000.0,32000.0,2019-09-02,128265
4,"BadComedian - о Бондарчуке, Саше Грей и 10 луч...",BadComedian,63,16217778,461000.0,23000.0,2017-05-10,35355


In [161]:
df['likes/dislikes'] = df['likes']/df['dislikes']
df['views/likes'] = df['views']/df['likes']
df['views/comments'] = df['views']/df['comments']

In [162]:
df.head()

,full_title,guest,length,views,likes,dislikes,date,comments,likes/dislikes,views/likes,views/comments
0,"Долгополов - депрессия, политика, секс втроем",Долгополов,84,7461608,304000.0,36000.0,2019-11-25,40781,8.444444,24.5448,182.968
1,Егор Крид - уход из Black Star и звонок Попере...,Егор Крид,113,22334678,634000.0,50000.0,2019-05-28,68999,12.680000,35.2282,323.696
2,"Познер - о цензуре, страхе и Путине",Познер,78,12882333,338000.0,17000.0,2017-10-18,40944,19.882353,38.1134,314.633
3,Беслан. Помни,Беслан,190,21968899,1100000.0,32000.0,2019-09-02,128265,34.375000,19.9717,171.277
4,"BadComedian - о Бондарчуке, Саше Грей и 10 луч...",BadComedian,63,16217778,461000.0,23000.0,2017-05-10,35355,20.043478,35.1796,458.712


In [163]:
df.to_csv('vdud.csv', index=False)

In [164]:
datetime.datetime.today().date()

datetime.date(2020, 11, 3)